In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [20]:
from google.colab import files
uploaded = files.upload()

Saving Test.csv to Test (2).csv
Saving Train.csv to Train (1).csv


In [ ]:
df_train=pd.read_csv('Train (1).csv')
df_test=pd.read_csv('Test (2).csv')

In [267]:
df_train.shape

(29451, 12)

In [268]:
df_test.shape

(68720, 11)

In [269]:
### ADDRESS

In [322]:
def city(address):
 city_name=address.split(',')[-1]
 return city_name
def locality(address):
 locality=address.split(',')[-2]
 return locality
df_train['loc']=df_train['ADDRESS'].apply(lambda x : locality(x))
df_train['City']=df_train['ADDRESS'].apply(lambda x : city(x))
df_test['loc']=df_test['ADDRESS'].apply(lambda x : locality(x))
df_test['City']=df_test['ADDRESS'].apply(lambda x : city(x))

In [323]:
df_train.drop(columns=['ADDRESS'],inplace=True)
df_test.drop(columns=['ADDRESS'],inplace=True)

In [324]:
Encoding = df_train.groupby('City')['TARGET(PRICE_IN_LACS)'].mean()
df_train['City_mean']= df_train.City.map(Encoding )
df_test['City_mean']= df_test.City.map(Encoding )

In [325]:
df_test.isnull().sum()

POSTED_BY               0
UNDER_CONSTRUCTION      0
RERA                    0
BHK_NO.                 0
BHK_OR_RK               0
SQUARE_FT               0
READY_TO_MOVE           0
RESALE                  0
LONGITUDE               0
LATITUDE                0
loc                     0
City                    0
City_mean             125
dtype: int64

In [326]:
df_train.corr()

,UNDER_CONSTRUCTION,RERA,BHK_NO.,SQUARE_FT,READY_TO_MOVE,RESALE,LONGITUDE,LATITUDE,TARGET(PRICE_IN_LACS),City_mean
UNDER_CONSTRUCTION,1.000000,0.363826,-0.040712,-0.004204,-1.000000,-0.347405,0.006440,-0.000381,0.055399,0.134327
RERA,0.363826,1.000000,0.009547,-0.006229,-0.363826,-0.270351,0.104976,-0.065106,0.067636,0.206481
BHK_NO.,-0.040712,0.009547,1.000000,0.005303,0.040712,0.014581,0.068730,0.046930,0.112283,-0.051758
SQUARE_FT,-0.004204,-0.006229,0.005303,1.000000,0.004204,0.001732,-0.012591,0.000803,0.402685,0.002244
READY_TO_MOVE,-1.000000,-0.363826,0.040712,0.004204,1.000000,0.347405,-0.006440,0.000381,-0.055399,-0.134327
RESALE,-0.347405,-0.270351,0.014581,0.001732,0.347405,1.000000,0.024038,0.014844,-0.207378,-0.167869
LONGITUDE,0.006440,0.104976,0.068730,-0.012591,-0.006440,0.024038,1.000000,-0.155062,-0.031112,-0.197449
LATITUDE,-0.000381,-0.065106,0.046930,0.000803,0.000381,0.014844,-0.155062,1.000000,-0.017254,-0.112636
TARGET(PRICE_IN_LACS),0.055399,0.067636,0.112283,0.402685,-0.055399,-0.207378,-0.031112,-0.017254,1.000000,0.133150
City_mean,0.134327,0.206481,-0.051758,0.002244,-0.134327,-0.167869,-0.197449,-0.112636,0.133150,1.000000


In [327]:
df_train.drop(columns=['READY_TO_MOVE'],inplace=True)
df_test.drop(columns=['READY_TO_MOVE'],inplace=True)

In [328]:
df_train

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO.,BHK_OR_RK,SQUARE_FT,RESALE,LONGITUDE,LATITUDE,TARGET(PRICE_IN_LACS),loc,City,City_mean
0,Owner,0,0,2,BHK,1300.236407,1,12.969910,77.597960,55.0,Ksfc Layout,Bangalore,162.827783
1,Dealer,0,0,2,BHK,1275.000000,1,12.274538,76.644605,51.0,Vishweshwara Nagar,Mysore,62.353488
2,Owner,0,0,2,BHK,933.159722,1,12.778033,77.632191,43.0,Jigani,Bangalore,162.827783
3,Owner,0,1,2,BHK,929.921143,1,28.642300,77.344500,62.5,Sector-1 Vaishali,Ghaziabad,54.406964
4,Dealer,1,0,2,BHK,999.009247,1,22.592200,88.484911,60.5,New Town,Kolkata,99.807080
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29446,Owner,0,0,3,BHK,2500.000000,1,27.140626,78.043277,45.0,Shamshabad Road,Agra,39.883133
29447,Owner,0,0,2,BHK,769.230769,1,39.945409,-86.150721,16.0,,Vapi,24.427027
29448,Dealer,0,0,2,BHK,1022.641509,1,26.928785,75.828002,27.1,Ajmer Road,Jaipur,55.332848
29449,Owner,0,0,2,BHK,927.079009,1,12.900150,80.227910,67.0,Sholinganallur,Chennai,141.673147


In [329]:
df_train.drop(columns=['loc','City'],inplace=True)
df_test.drop(columns=['loc','City'],inplace=True)

In [330]:
df_train['POSTED_BY'].value_counts()

Dealer     18291
Owner      10538
Builder      622
Name: POSTED_BY, dtype: int64

In [331]:
df_train['POSTED_BY_enc']=df_train['POSTED_BY'].map({'Dealer':0,'Owner':1,'Builder':2})

In [332]:
df_test['POSTED_BY_enc']=df_test['POSTED_BY'].map({'Dealer':0,'Owner':1,'Builder':2})

In [333]:
df_train.drop(columns=['POSTED_BY'],inplace=True)
df_test.drop(columns=['POSTED_BY'],inplace=True)

In [334]:
df_train['BHK_OR_RK']=df_train['BHK_OR_RK'].map({'BHK':0,'RK':1})
df_test['BHK_OR_RK']=df_test['BHK_OR_RK'].map({'BHK':0,'RK':1})

In [335]:
Encoding = df_train.groupby('LATITUDE')['TARGET(PRICE_IN_LACS)'].mean()
df_train['lat_mean']= df_train.LATITUDE.map(Encoding )
df_test['lat_mean']= df_test.LATITUDE.map(Encoding )

In [336]:
Encoding = df_train.groupby('LONGITUDE')['TARGET(PRICE_IN_LACS)'].mean()
df_train['lon_mean']= df_train.LONGITUDE.map(Encoding )
df_test['lon_mean']= df_test.LONGITUDE.map(Encoding )

In [337]:
df_test.isnull().sum()

UNDER_CONSTRUCTION       0
RERA                     0
BHK_NO.                  0
BHK_OR_RK                0
SQUARE_FT                0
RESALE                   0
LONGITUDE                0
LATITUDE                 0
City_mean              125
POSTED_BY_enc            0
lat_mean              4333
lon_mean              4313
dtype: int64

In [338]:
df_test_col=df_test.columns
from sklearn.neighbors import KNeighborsRegressor
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
it=IterativeImputer(estimator=KNeighborsRegressor(n_neighbors=5),random_state=0)
df_test= pd.DataFrame(it.fit_transform(df_test),columns=df_test_col)

In [339]:
df_train.drop(columns=['LATITUDE','LONGITUDE'],inplace=True)

In [340]:
df_train_encoded=pd.get_dummies(df_train,drop_first=True)
df_test_encoded=pd.get_dummies(df_test,drop_first=True)

In [341]:
df_test_encoded_columns=df_test_encoded.columns
df_test_encoded.isnull().sum()

UNDER_CONSTRUCTION    0
RERA                  0
BHK_NO.               0
BHK_OR_RK             0
SQUARE_FT             0
RESALE                0
LONGITUDE             0
LATITUDE              0
City_mean             0
POSTED_BY_enc         0
lat_mean              0
lon_mean              0
dtype: int64

In [286]:
df_test_encoded.loc[df_test_encoded['City_mean'].isnull(),'City_mean']=df_train['TARGET(PRICE_IN_LACS)'].mean()

In [287]:
df_test_encoded.isnull().sum()

UNDER_CONSTRUCTION    0
RERA                  0
BHK_NO.               0
BHK_OR_RK             0
SQUARE_FT             0
RESALE                0
LONGITUDE             0
LATITUDE              0
City_mean             0
POSTED_BY_enc         0
dtype: int64

In [37]:
df_train_encoded['BHK_NO.'].value_counts()

2     13324
3     10546
1      3574
4      1723
5       190
6        52
7        11
8        10
15        4
10        4
20        4
12        3
9         3
13        1
11        1
17        1
Name: BHK_NO., dtype: int64

In [342]:
### Model Building
### Bagging Regressor
df_train_encoded.corr()

,UNDER_CONSTRUCTION,RERA,BHK_NO.,BHK_OR_RK,SQUARE_FT,RESALE,TARGET(PRICE_IN_LACS),City_mean,POSTED_BY_enc,lat_mean,lon_mean
UNDER_CONSTRUCTION,1.000000,0.363826,-0.040712,0.020719,-0.004204,-0.347405,0.055399,0.134327,-0.103676,0.072826,0.072486
RERA,0.363826,1.000000,0.009547,0.006056,-0.006229,-0.270351,0.067636,0.206481,-0.185657,0.085190,0.084318
BHK_NO.,-0.040712,0.009547,1.000000,-0.045231,0.005303,0.014581,0.112283,-0.051758,-0.090140,0.091775,0.091574
BHK_OR_RK,0.020719,0.006056,-0.045231,1.000000,-0.000292,-0.029346,-0.004429,0.035290,0.003133,0.003163,0.003159
SQUARE_FT,-0.004204,-0.006229,0.005303,-0.000292,1.000000,0.001732,0.402685,0.002244,0.004265,0.389038,0.388453
RESALE,-0.347405,-0.270351,0.014581,-0.029346,0.001732,1.000000,-0.207378,-0.167869,-0.107234,-0.100621,-0.099802
TARGET(PRICE_IN_LACS),0.055399,0.067636,0.112283,-0.004429,0.402685,-0.207378,1.000000,0.133150,-0.071863,0.432742,0.433393
City_mean,0.134327,0.206481,-0.051758,0.035290,0.002244,-0.167869,0.133150,1.000000,-0.385386,0.295863,0.295058
POSTED_BY_enc,-0.103676,-0.185657,-0.090140,0.003133,0.004265,-0.107234,-0.071863,-0.385386,1.000000,-0.144416,-0.143987
lat_mean,0.072826,0.085190,0.091775,0.003163,0.389038,-0.100621,0.432742,0.295863,-0.144416,1.000000,0.997407


In [ ]:
df_train_encoded.shape

(29451, 11)

In [289]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.decomposition import KernelPCA,IncrementalPCA

x=df_train_encoded.drop(columns=['TARGET(PRICE_IN_LACS)'])
y=np.log(df_train_encoded['TARGET(PRICE_IN_LACS)'])
xtrain,xtest,ytrain,ytest=train_test_split(x,y,random_state=0,test_size=0.2)
 
pipebr=Pipeline((
    ("pt",PowerTransformer()),
    ("ss",StandardScaler()),
    #("ipca",IncrementalPCA(n_components=10, batch_size=10),
    ("gbr", BaggingRegressor(random_state=0,n_estimators=250))
))

pipebr.fit(xtrain,ytrain)
ypred_train=np.abs(pipebr.predict(xtrain))
ypred_test=np.abs(pipebr.predict(xtest))

print("Testing Accuracy R2 score")
print(pipebr.score(xtest,ytest))
print("Training Accuracy R2 score")
print(pipebr.score(xtrain,ytrain))

print("Testing Accuracy RMSLE score")
print(np.sqrt(mean_squared_error(ytest,ypred_test)))
print("Training Accuracy RMSLE score")
print(np.sqrt(mean_squared_error( ytrain,ypred_train)))

Testing Accuracy R2 score
0.805958963390237
Training Accuracy R2 score
0.9691896935267259
Testing Accuracy RMSLE score
0.3882614371631682
Training Accuracy RMSLE score
0.1586508661850986


In [ ]:
from sklearn.model_selection import cross_val_score
be=[]
ve=[]
for i in range(115,120):
  dt=BaggingRegressor(n_estimators=i,random_state=0)
  score=cross_val_score(dt,xtrain,ytrain,cv=5,scoring='neg_mean_squared_log_error')
  print('n_estimator:',i)
  print('Bias Error',np.mean(np.sqrt(np.abs(score))))
  print('Variance Error',np.std(np.sqrt(np.abs(score)))/np.mean(np.sqrt(np.abs(score))))
  be.append(np.mean(np.sqrt(np.abs(score))))
  ve.append(np.std(np.sqrt(np.abs(score)))/np.mean(np.sqrt(np.abs(score))))

In [ ]:
### Random Forest Regressor

In [290]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error,mean_squared_error

x=df_train_encoded.drop(columns=['TARGET(PRICE_IN_LACS)'])
y=np.log(df_train_encoded['TARGET(PRICE_IN_LACS)'])
xtrain,xtest,ytrain,ytest=train_test_split(x,y,random_state=0,test_size=0.2)
 
pipebr=Pipeline((
    ("pt",PowerTransformer()),
    ("ss",StandardScaler()),
    ("gbr", RandomForestRegressor(random_state=0,n_estimators=285))
))

pipebr.fit(xtrain,ytrain)
ypred_train=np.abs(pipebr.predict(xtrain))
ypred_test=np.abs(pipebr.predict(xtest))

print("Testing Accuracy R2 score")
print(pipebr.score(xtest,ytest))
print("Training Accuracy R2 score")
print(pipebr.score(xtrain,ytrain))

print("Testing Accuracy RMSLE score")
print(np.sqrt(mean_squared_error(ytest,ypred_test)))
print("Training Accuracy RMSLE score")
print(np.sqrt(mean_squared_error( ytrain,ypred_train)))

Testing Accuracy R2 score
0.8063263946333343
Training Accuracy R2 score
0.9692566545551181
Testing Accuracy RMSLE score
0.38789366191554225
Training Accuracy RMSLE score
0.1584783719013614


In [325]:
from sklearn.model_selection import cross_val_score
be=[]
ve=[]
for i in [5,10,20,40,60,80,100,120,150,200,250,300,400]:
  dt=RandomForestRegressor(n_estimators=i,random_state=0)
  score=cross_val_score(dt,xtrain,ytrain,cv=5,scoring='neg_mean_squared_error')
  print('n_estimator:',i)
  print('Bias Error',np.mean(np.sqrt(np.abs(score))))
  print('Variance Error',np.std(np.sqrt(np.abs(score)))/np.mean(np.sqrt(np.abs(score))))
  be.append(np.mean(np.sqrt(np.abs(score))))
  ve.append(np.std(np.sqrt(np.abs(score)))/np.mean(np.sqrt(np.abs(score))))

n_estimator: 5
Bias Error 0.3647060739923873
Variance Error 0.02568998650298624
n_estimator: 10
Bias Error 0.3499146873004041
Variance Error 0.025505527427457427
n_estimator: 20
Bias Error 0.3422852191994978
Variance Error 0.028398450790622802
n_estimator: 40
Bias Error 0.33736105304122677
Variance Error 0.029908775970033594
n_estimator: 60
Bias Error 0.3366278701994273
Variance Error 0.030218804230109902
n_estimator: 80
Bias Error 0.33559139729156284
Variance Error 0.030101210328383584
n_estimator: 100
Bias Error 0.33559127879644335
Variance Error 0.030181116177091082
n_estimator: 120
Bias Error 0.33554620352087194
Variance Error 0.029097155523035892
n_estimator: 150
Bias Error 0.33511895721918294
Variance Error 0.029445916606523147
n_estimator: 200
Bias Error 0.3348904245026079
Variance Error 0.02941439510928082
n_estimator: 250
Bias Error 0.3346881462969945
Variance Error 0.029384272551650697
n_estimator: 300
Bias Error 0.3346221396021998
Variance Error 0.02962597559997254
n_estimat

In [ ]:
### Gradient Boosting Regressor

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_log_error,mean_squared_error

x=df_train_encoded.drop(columns=['TARGET(PRICE_IN_LACS)'])
y=np.log(df_train_encoded['TARGET(PRICE_IN_LACS)'])
xtrain,xtest,ytrain,ytest=train_test_split(x,y,random_state=0,test_size=0.2)
 
pipebr=Pipeline((
    ("pt",PowerTransformer()),
    ("ss",StandardScaler()),
    ("gbr", GradientBoostingRegressor(random_state=0,n_estimators=3500))
))

pipebr.fit(xtrain,ytrain)
ypred_train=np.abs(pipebr.predict(xtrain))
ypred_test=np.abs(pipebr.predict(xtest))

print("Testing Accuracy R2 score")
print(pipebr.score(xtest,ytest))
print("Training Accuracy R2 score")
print(pipebr.score(xtrain,ytrain))

print("Testing Accuracy RMSLE score")
print(np.sqrt(mean_squared_error(ytest,ypred_test)))
print("Training Accuracy RMSLE score")
print(np.sqrt(mean_squared_error( ytrain,ypred_train)))

In [ ]:
### XG boost

In [166]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer,StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_log_error

x=df_train_encoded.drop(columns=['TARGET(PRICE_IN_LACS)'])
y=np.log(df_train_encoded['TARGET(PRICE_IN_LACS)'])
xtrain,xtest,ytrain,ytest=train_test_split(x,y,random_state=0,test_size=0.2)
 
pipebr=Pipeline((
    ("pt",PowerTransformer()),
    ("ss",StandardScaler()),
    ("gbr", XGBRegressor(random_state=0,n_estimators=3000))
))

pipebr.fit(xtrain,ytrain)
ypred_train=np.abs(pipebr.predict(xtrain))
ypred_test=np.abs(pipebr.predict(xtest))

print("Testing Accuracy R2 score")
print(pipebr.score(xtest,ytest))
print("Training Accuracy R2 score")
print(pipebr.score(xtrain,ytrain))

print("Testing Accuracy RMSLE score")
print(np.sqrt(mean_squared_error(ytest,ypred_test)))
print("Training Accuracy RMSLE score")
print(np.sqrt(mean_squared_error( ytrain,ypred_train)))

[08:52:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Testing Accuracy R2 score
0.918534935511386
Training Accuracy R2 score
0.9592952445300659
Testing Accuracy RMSLE score
0.24813760801846205
Training Accuracy RMSLE score
0.18374221633562307


In [ ]:
### Decision Tree Regressor

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_log_error

x=df_train_encoded.drop(columns=['TARGET(PRICE_IN_LACS)'])
y=df_train_encoded['TARGET(PRICE_IN_LACS)']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,random_state=0,test_size=0.2)
 
pipebr=Pipeline((
    ("pt",PowerTransformer()),
    ("ss",StandardScaler()),
    ("gbr", DecisionTreeRegressor(random_state=0,max_depth=26))
))

pipebr.fit(xtrain,ytrain)
ypred_train=np.abs(pipebr.predict(xtrain))
ypred_test=np.abs(pipebr.predict(xtest))

print("Testing Accuracy R2 score")
print(pipebr.score(xtest,ytest))
print("Training Accuracy R2 score")
print(pipebr.score(xtrain,ytrain))

print("Testing Accuracy RMSLE score")
print(np.sqrt(mean_squared_log_error(ytest,ypred_test)))
print("Training Accuracy RMSLE score")
print(np.sqrt(mean_squared_log_error( ytrain,ypred_train)))

Testing Accuracy R2 score
0.935884963855968
Training Accuracy R2 score
0.999786759299274
Testing Accuracy RMSLE score
0.4106158941940213
Training Accuracy RMSLE score
0.08618494097804878


In [167]:
ypredtrain =pipebr.predict(df_test_encoded)
ypredtrain=np.abs((np.exp(ypredtrain)-1))

In [359]:
#ypredtrain=np.abs(pipebr.predict(df_test_encoded))

In [168]:
file1=pd.DataFrame(ypredtrain,columns=['TARGET(PRICE_IN_LACS)'])

In [169]:
file1.to_csv('sample_submission.csv', index=False)

In [170]:
from google.colab import files
files.download('sample_submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>